## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,61.54,64,12,20.69,few clouds
1,1,Hilo,US,19.7297,-155.0900,76.28,92,90,5.75,light rain
2,2,Aklavik,CA,68.2191,-135.0107,28.47,83,100,9.93,overcast clouds
3,3,College,US,64.8569,-147.8028,35.26,94,75,6.91,fog
4,4,Tarata,BO,-17.6167,-66.0167,85.42,29,46,9.13,scattered clouds


In [3]:
# Verify lat and lng are float64
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,61.54,64,12,20.69,few clouds
1,1,Hilo,US,19.7297,-155.0900,76.28,92,90,5.75,light rain
4,4,Tarata,BO,-17.6167,-66.0167,85.42,29,46,9.13,scattered clouds
5,5,Arraial Do Cabo,BR,-22.9661,-42.0278,67.96,94,90,12.66,light intensity drizzle
7,7,Punta Arenas,CL,-53.1500,-70.9167,53.71,35,20,31.07,few clouds
8,8,Ribeira Grande,PT,38.5167,-28.7000,72.00,94,75,12.66,broken clouds
9,9,Sarandi,BR,-23.4436,-51.8739,65.17,83,100,11.52,overcast clouds
10,10,Ponta Do Sol,PT,32.6667,-17.1000,76.01,70,16,3.85,few clouds
11,11,Hermanus,ZA,-34.4187,19.2345,65.89,66,89,7.76,overcast clouds
12,12,Port Elizabeth,ZA,-33.9180,25.5701,61.11,68,1,18.28,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                520
City                   520
Country                517
Lat                    520
Lng                    520
Max Temp               520
Humidity               520
Cloudiness             520
Wind Speed             520
Current Description    520
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                517
City                   517
Country                517
Lat                    517
Lng                    517
Max Temp               517
Humidity               517
Cloudiness             517
Wind Speed             517
Current Description    517
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,61.54,few clouds,-33.5906,26.8910,
1,Hilo,US,76.28,light rain,19.7297,-155.0900,
4,Tarata,BO,85.42,scattered clouds,-17.6167,-66.0167,
5,Arraial Do Cabo,BR,67.96,light intensity drizzle,-22.9661,-42.0278,
7,Punta Arenas,CL,53.71,few clouds,-53.1500,-70.9167,
8,Ribeira Grande,PT,72.00,broken clouds,38.5167,-28.7000,
9,Sarandi,BR,65.17,overcast clouds,-23.4436,-51.8739,
10,Ponta Do Sol,PT,76.01,few clouds,32.6667,-17.1000,
11,Hermanus,ZA,65.89,overcast clouds,-34.4187,19.2345,
12,Port Elizabeth,ZA,61.11,clear sky,-33.9180,25.5701,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key,
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    #print("Hotel found.")
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,61.54,few clouds,-33.5906,26.8910,The Halyards Hotel
1,Hilo,US,76.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Tarata,BO,85.42,scattered clouds,-17.6167,-66.0167,Hacienda castillo
5,Arraial Do Cabo,BR,67.96,light intensity drizzle,-22.9661,-42.0278,Pousada Porto Praia
7,Punta Arenas,CL,53.71,few clouds,-53.1500,-70.9167,Hotel Dreams Del Estrecho


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name']!='']
clean_hotel_df.count()

City                   480
Country                480
Max Temp               480
Current Description    480
Lat                    480
Lng                    480
Hotel Name             480
dtype: int64

In [12]:
# Print hotels in cities with hotel names.
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,61.54,few clouds,-33.5906,26.8910,The Halyards Hotel
1,Hilo,US,76.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Tarata,BO,85.42,scattered clouds,-17.6167,-66.0167,Hacienda castillo
5,Arraial Do Cabo,BR,67.96,light intensity drizzle,-22.9661,-42.0278,Pousada Porto Praia
7,Punta Arenas,CL,53.71,few clouds,-53.1500,-70.9167,Hotel Dreams Del Estrecho


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt>{Current Description} and {Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))